In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-chandigarh: Message: 
Stacktrace:
0   chromedriver                        0x0000000102b1b9a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x0000000102b13e10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000102718be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x000000010275cf10 cxxbridge1$string$len + 368852
4   chromedriver                        0x00000001027964a4 cxxbridge1$string$len + 603752
5   chromedriver                        0x0000000102751a08 cxxbridge1$string$len + 322508
6   chromedriver                        0x000000010275266c cxxbridge1$string$len + 325680
7   chromedriver                        0x0000000102ae2068 cxxbridge1$str$ptr + 1651256
8   chromedriver                        0x0000000102ae699c cxxbridge1$str$ptr + 1669996
9   chromedriver                        0x0000000102ac71fc cxxbridge1$str$pt

NameError: name 'driver' is not defined

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165681,HVAC Seater (2+3),07:00,05h 35m,12:35,4.3,INR 404.76,46 Seats available
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165684,HVAC Seater (2+3),07:55,05h 35m,13:30,4.6,INR 404.76,47 Seats available
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165687,HVAC Seater (2+3),08:25,05h 35m,14:00,4.1,INR 404.76,47 Seats available
3,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165676,HVAC Seater (2+3),09:25,05h 35m,15:00,4.0,INR 404.76,47 Seats available
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165689,HVAC Seater (2+3),10:30,05h 35m,16:05,3.7,INR 404.76,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
818,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1564,Ordinary 3+2 Non AC Seater,12:15,05h 15m,17:30,3.9,INR 300,27 Seats available
819,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1285,Ordinary 3+2 Non AC Seater,18:40,05h 20m,00:00,3.9,INR 303,37 Seats available
820,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 573,Ordinary 3+2 Non AC Seater,19:40,04h 35m,00:15,3.9,INR 318,37 Seats available
821,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165783,HVAC Seater (2+3),07:55,05h 35m,13:30,3.4,INR 406.67,47 Seats available
